In [9]:
import os
import uuid
import csv
import asyncio
from datetime import datetime
from supabase import create_async_client, AsyncClient
from dotenv import load_dotenv

load_dotenv(override=True)

SUPABASE_URL = os.getenv("SUPABASE_URL", "")
SUPABASE_API_KEY = os.getenv("SUPABASE_API_KEY", "")

async def get_client() -> AsyncClient:
    """
    Helper function to create a new asynchronous Supabase client.
    """
    return await create_async_client(SUPABASE_URL, SUPABASE_API_KEY)

In [10]:
async def set_status(id: str, status: str) -> dict:
    """
    Updates the status of a trip in the 'trips' table.
    
    Parameters:
        trip_id (str): The unique identifier of the trip.
        status (str): The new status, must be one of: "arrived", "delayed", "in progress".
        
    Returns:
        dict: A response dict indicating success or error with corresponding data.
    """
    allowed_statuses = {"arrived", "delayed", "in progress"}
    if status not in allowed_statuses:
        return {
            "status": "error",
            "message": f"Invalid status provided. Allowed statuses: {allowed_statuses}"
        }

    client = await get_client()
    response = await client.table("trips").update({"status": status}).eq("id", id).execute()
    result = response.dict()

    if result.get("error") is None:
        return {"status": "success", "data": result.get("data")}
    else:
        return {"status": "error", "message": result.get("error")}

In [11]:
status = await set_status("2", "arrived")
print(status)

{'status': 'success', 'data': [{'id': 2, 'created_at': '2025-03-02T08:13:55.509655+00:00', 'user_id': 9, 'start_location': 'Los Angeles', 'destination': 'San Francisco', 'eta': None, 'status': 'arrived', 'interval': 30}]}


/var/folders/2s/w7mvcjhx01q7sjwc9k_nc7xw0000gn/T/ipykernel_35582/2152048223.py:21: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  result = response.dict()
